# ShowCase Examples
this is a file that contains codes that were removed but are importat, since they were used to test some theories

In [1]:
from utils.utils import *
from utils.ee_utils import *
from utils.utils import TextColors as tc
import ee
import geemap
from rois import ROIS
rois = ROIS()
from IPython.display import display

In [2]:
ee.Initialize()
Map = geemap.Map()

In [3]:
vis_params_s1_bright = {
    'min': -10,
    'max': 10,
    'bands': ['VV_db']
}

## High cloud cover results in false snow_cover result in sentinel 2 images

For exapmle in this case, we are testing an image in summer in paris, but as you can see we are seeing a high snow/ice coverege and that is because there is a high snow covege.

<span style="color: #FFFF11;">This is the reason we are not using snow removal on summer dataset anymore.</span>

If you want to use snow removal, you need to <span style="color: #FF4444;">make sure that the `cloud_cover` is low in the scene.</span>
- in our case we are using our custom cloud_cover check, which only checks our `roi`, so the whole scene could still be cloudy, and thus wrong snow_cover.

In [4]:
roi = rois.bercy

s2_test_col = ee.ImageCollection('COPERNICUS/S2_SR') \
                    .filterDate('2021-05-17', '2021-05-19') \
                    .filterBounds(roi)
                    
sen2_print(s2_test_col)
s2_test_img = s2_test_col.first()

scl = s2_test_img.select('SCL')
mask = scl.eq(11)

Map = geemap.Map()
Map.addLayer(s2_test_img, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 Image', True)
Map.addLayer(mask.updateMask(mask), {'palette': 'blue'}, 'Water Mask')
Map.centerObject(roi, zoom=10)
Map.addLayerControl()
Map


Map(center=[48.843641547684896, 2.380256652833762], controls=(WidgetControl(options=['position', 'transparent_…

## Sentinel 1A and 1B, Tandem Orbit 
sentinel 1a and 1b orbits result in a pretty simmilar, becuse of the short Spatial BaseLine.

- Although I still don't know, how can you determine whether two, realtive orbit numbers are tandem or not. -> <span style="color: #FF1111; text-decoration: line-through;">More Research Needed</span>

<span style="color: #11FF11;">Found The Answer - Read The Cells Below</span>


In [5]:
# Overwrite the default print function to for a consicise print
sen1_print = lambda s2_collection: ee_property_printer(s2_collection, propertie_name_list=['system:time_start','platform_number',
                                                                                                    'relativeOrbitNumber_start', 'orbitProperties_pass',
                                                                                                    'cycleNumber'], first_is_t_in_millis=True)

In [14]:
roi = rois.paris
# s2_col,s1_col = s1s2(roi,date = ('2021-07-15', '2021-08-15'),priority_path = 'DESCENDING',max_cloud = 100,max_snow = 100,month_span = 1,retry_days=0)
s1_col = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate('2021-06-24', '2021-08-20') \
    .filterBounds(roi) \
    .filter(ee.Filter.eq('resolution','H')) \
    .filter(ee.Filter.eq('instrumentMode','IW'))\
    .filter(ee.Filter.contains('.geo', roi)) \
    .filterMetadata('orbitProperties_pass', 'equals', 'DESCENDING') 
    
s1_col.size().getInfo()

20

### Finding the most occured orbit
well here both orbit `110` and `8` have equal passes.
But in other cases we would like to choose images that have more passes over the roi, to get a better `spackle` mitigation.

In [15]:
rel_list = s1_col.aggregate_array('relativeOrbitNumber_start')
x = find_most_repeated_element(rel_list)
x


There is a pattern `BA AB BA AB BA...` with each `BA` differing only **1** day and each `BA AB` having a **5** day difference

In [16]:
sen1_print(s1_col.sort('system:time_start'))

,Property,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,system:time_start,2021-06-25,2021-06-26,2021-07-01,2021-07-02,2021-07-07,2021-07-08,2021-07-13,2021-07-14,2021-07-19,2021-07-20,2021-07-25,2021-07-26,2021-07-31,2021-08-01,2021-08-06,2021-08-07,2021-08-12,2021-08-13,2021-08-18,2021-08-19
1,platform_number,B,A,A,B,B,A,A,B,B,A,A,B,B,A,A,B,B,A,A,B
2,relativeOrbitNumber_start,8,110,8,110,8,110,8,110,8,110,8,110,8,110,8,110,8,110,8,110
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,164,234,235,164,165,235,236,165,166,236,237,166,167,237,238,167,168,238,239,168


<span style="color: #dc0fff;">Let's devide this imageCollection, into two collections of `Only 1A` and `Only 1B`</span>

Hmmm... as you can see, the `1A` collection has `two` diffrent `relativeOrbitnumber`s which mean there are two diffrent scences covering this roi, form two diffrent angles!






In [17]:
sen1_a =s1_col.filter(ee.Filter.eq('platform_number', 'A')).sort('system:time_start')
sen1_print(sen1_a)

,Property,0,1,2,3,4,5,6,7,8,9
0,system:time_start,2021-06-26,2021-07-01,2021-07-08,2021-07-13,2021-07-20,2021-07-25,2021-08-01,2021-08-06,2021-08-13,2021-08-18
1,platform_number,A,A,A,A,A,A,A,A,A,A
2,relativeOrbitNumber_start,110,8,110,8,110,8,110,8,110,8
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,234,235,235,236,236,237,237,238,238,239


Let's visualize this:

In [ ]:
sen1a_110 = toDb(sen1_a.filter(ee.Filter.eq('relativeOrbitNumber_start', 110)).sort('system:time_start').map(toLinear).mean().clip(roi))
sen1a_8 = toDb(sen1_a.filter(ee.Filter.eq('relativeOrbitNumber_start', 8)).sort('system:time_start').map(toLinear).mean().clip(roi))
left_layer = geemap.ee_tile_layer(sen1a_110, vis_params_s1_bright, 'sen1a_110')
right_layer = geemap.ee_tile_layer(sen1a_8, vis_params_s1_bright, 'sen1a_8')
Map = geemap.Map(zoom=12)
Map.centerObject(roi)
Map.split_map(left_layer, right_layer)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

### Average Viewing angle of each Image in the roi
Here we can see that orbit `110` has a much steeper `incident angle`, in a case where we have oribts with equal passes over the roi, we want to choose the images that have a higher incidetn angle, thus finder grain detail.

In [ ]:
display(get_band_average(sen1a_8,'angle'),
        get_band_average(sen1a_110,'angle'))

We can also see that `sentinel-1b`, of the same orbits have almost `the same` incident angle as `1a`

In [ ]:
sen1_b =s1_col.filter(ee.Filter.eq('platform_number', 'B')).sort('system:time_start')
sen1b_110 = toDb(sen1_a.filter(ee.Filter.eq('relativeOrbitNumber_start', 110)).sort('system:time_start').map(toLinear).mean().clip(roi))
sen1b_8 = toDb(sen1_a.filter(ee.Filter.eq('relativeOrbitNumber_start', 8)).sort('system:time_start').map(toLinear).mean().clip(roi))
display(get_band_average(sen1b_8,'angle'),
        get_band_average(sen1b_110,'angle'))

The same for 1B

In [ ]:
sen1_print(s1_col.filter(ee.Filter.eq('platform_number', 'B')).sort('system:time_start'))

,Property,0,1,2,3,4,5,6,7,8,9
0,system:time_start,2021-06-25,2021-07-02,2021-07-07,2021-07-14,2021-07-19,2021-07-26,2021-07-31,2021-08-07,2021-08-12,2021-08-19
1,platform_number,B,B,B,B,B,B,B,B,B,B
2,relativeOrbitNumber_start,8,110,8,110,8,110,8,110,8,110
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,164,164,165,165,166,166,167,167,168,168


### Let's deivde them by Oribt Number
`1A` and `1B` have the same Orbit Number over an Area!
- This is the Tandem Orbit
- So for consistat viewing geometry, filter by orbit number

In [ ]:
print(tc.BOLD,'Relative orbit number 8: ',tc.ENDC)
sen1_print(s1_col.filter(ee.Filter.eq('relativeOrbitNumber_start', ee.Number(8))).sort('system:time_start'))


 Relative orbit number 8:  


,Property,0,1,2,3,4,5,6,7,8,9
0,system:time_start,2021-06-25,2021-07-01,2021-07-07,2021-07-13,2021-07-19,2021-07-25,2021-07-31,2021-08-06,2021-08-12,2021-08-18
1,platform_number,B,A,B,A,B,A,B,A,B,A
2,relativeOrbitNumber_start,8,8,8,8,8,8,8,8,8,8
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,164,235,165,236,166,237,167,238,168,239


In [ ]:
print(tc.BOLD,'Relative orbit number 110: ',tc.ENDC)
sen1_print(s1_col.filter(ee.Filter.eq('relativeOrbitNumber_start', ee.Number(110))).sort('system:time_start'))

 Relative orbit number 110:  


,Property,0,1,2,3,4,5,6,7,8,9
0,system:time_start,2021-06-26,2021-07-02,2021-07-08,2021-07-14,2021-07-20,2021-07-26,2021-08-01,2021-08-07,2021-08-13,2021-08-19
1,platform_number,A,B,A,B,A,B,A,B,A,B
2,relativeOrbitNumber_start,110,110,110,110,110,110,110,110,110,110
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,234,164,235,165,236,166,237,167,238,168


### Comparing 1A and 1B on the same Orbit
lest filter the collection by only `orbit==8`, and then devide it into `1A` and `1B`

In [ ]:
s1_col_8 = s1_col.filter(ee.Filter.eq('relativeOrbitNumber_start', ee.Number(8))).sort('system:time_start').map(toLinear)

In [ ]:
s1a = toDb(s1_col_8.filter(ee.Filter.eq('platform_number', 'A')).mean().clip(roi))
s1b = toDb(s1_col_8.filter(ee.Filter.eq('platform_number', 'B')).mean().clip(roi))
left_layer = geemap.ee_tile_layer(s1a, vis_params_s1_bright, 'S1a')
right_layer = geemap.ee_tile_layer(s1b, vis_params_s1_bright, 'S1b')

<span style="color: #11FF11;">The Diffrence is a lot Less!</span>

- Corner reflectors stay a corner reflector! unlike diffrent orbits, these are `directional corner reflectors` the since the the dirrection in orbits change they might be a CR in one of them and not in the other!

- The spacle is higher since we devided the collection both by `A` and `B` and `relativeOrbit`

But we made out point!

Which  is: <span style="font-weight: bold; color: red;">USE ONLY ONE ORBIT</span>.

In [ ]:
Map = geemap.Map(zoom=12)
Map.centerObject(roi)
Map.split_map(left_layer, right_layer)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Viewing Angle

In [ ]:
# Define an area of interest (AOI)
aoi = ee.Geometry.Point([-122.4439, 37.7538])

# Load the Sentinel-1 data collection
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate('2022-01-01', '2022-01-31') \
    .first()

# Select the 'angle' band
angle = sentinel1.select('angle')

# Define the visualization parameters
vis_params = {
    'min': 25,
    'max': 45,
    'palette': ['white', 'red']
}

# Display the image using geemap
Map = geemap.Map(center=[37.7538, -122.4439], zoom=10)
Map.addLayer(angle, vis_params, 'Angle')
#Map.addLayerControl()
#Map.setControlVisibility(layer_control=True, fullscreen_control=True, lat_lng_popup=True)
Map

Map(center=[37.7538, -122.4439], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

# Usage of new s1s2 funciton

In [18]:
s2_col,s1_col = s1s2(roi,date = ('2021-07-15', '2021-08-15'),priority_path = 'DESCENDING',max_cloud = 100,max_snow = 100,month_span = 1,retry_days=0)

◍◍Finding S2
◍Single scene coverege was fount!
◍◍Finding S1
mean date:  2021-07-21
final date range:  2021-06-21 2021-08-21
◍checking for DESCENDING single scene
Mosiac Covers 93.96914404374861 percent of the roi.
Mosaic Coverege Accepted
Mosaic Coverege Accepted


In [19]:
sen1_print(s1_col.sort('system:time_start'))

,Property,0,1,2,3,4,5,6,7,8,9
0,system:time_start,2021-06-26,2021-07-02,2021-07-08,2021-07-14,2021-07-20,2021-07-26,2021-08-01,2021-08-07,2021-08-13,2021-08-19
1,platform_number,A,B,A,B,A,B,A,B,A,B
2,relativeOrbitNumber_start,110,110,110,110,110,110,110,110,110,110
3,orbitProperties_pass,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING,DESCENDING
4,cycleNumber,234,164,235,165,236,166,237,167,238,168


In [20]:
sen2_print(s2_col.sort('system:time_start'))

,Property,0,1
0,system:time_start,2021-07-19,2021-07-22
1,roi_cloud_cover,1.729341,0.024054
2,CLOUDY_PIXEL_PERCENTAGE,2.355076,0.408007
3,CLOUD_SHADOW_PERCENTAGE,0.077020,0.250338
4,VEGETATION_PERCENTAGE,60.976559,59.382445
5,NOT_VEGETATED_PERCENTAGE,35.378841,38.356686
6,CLOUD_COVERAGE_ASSESSMENT,2.355076,0.408007
7,GENERATION_TIME,1626888199000,1626958595000
8,SENSING_ORBIT_NUMBER,51,94
9,NODATA_PIXEL_PERCENTAGE,0.000056,26.772150


In [16]:
class Bold_Background:
    BLACK = '\033[1;40m'
    RED = '\033[1;41m'
    GREEN = '\033[1;42m'
    YELLOW = '\033[1;43m'
    BLUE = '\033[1;44m'
    PURPLE = '\033[1;45m'
    CYAN = '\033[1;46m'
    WHITE = '\033[1;47m'

In [58]:
print(TextColors.High_INTENSITY_BACKGROUNDs.BLUE,"HELLO",'\033[0m')
print(TextColors.BOLD_BAKGROUNDs.PURPLE,"HELLO",'\033[0m')
print(TextColors.High_INTENSITYs.BLUE,"HELLO",'\033[0m')
print(TextColors.BLACK_TEXT_WIHT_BACKGROUNDs.GREEN,"HELLO",'\033[0m')


print('\033[48;2;50;50;100m',"HELLO",'\033[0m')


print('\033[38;2;165;165;0m',"HELLO",'\033[0m')


 HELLO 
 HELLO 
 HELLO 
 HELLO 
 HELLO 
 HELLO 


In [ ]:
TextColors.BOLDs.GREEN

In [ ]:
import numpy as np

def divide_image(image):
    height, width, _ = image.shape
    patch_size = 256
    patches = []
    for i in range(0, height, patch_size):
        for j in range(0, width, patch_size):
            patch = image[i:i+patch_size, j:j+patch_size]
            if patch.shape[0] == patch_size and patch.shape[1] == patch_size:
                patches.append(patch)
    return np.array(patches)

In [60]:
min_left_out = float('inf')
min_left_out

inf

# Testing best overlap

In [67]:
import numpy as np
def divide_image(img):

    # Get the width and height of the input image
    width, height = img.shape

    # Initialize variables to store the best overlap value and the minimum number of pixels left out
    best_overlap = None
    min_left_out = float('inf')

    # Try different overlap values
    for overlap in range(1, 256):
        # Calculate the number of patches needed in both the x and y directions with the current overlap value
        x_patches = (width - overlap) // (256 - overlap)
        y_patches = (height - overlap) // (256 - overlap)

        # Calculate the total number of pixels left out with the current overlap value
        left_out = (width - (x_patches * 256 - (x_patches - 1) * overlap)) * (height - (y_patches * 256 - (y_patches - 1) * overlap))

        # Check if the current overlap value is better than the previous best overlap value
        if left_out < min_left_out:
            best_overlap = overlap
            min_left_out = left_out

    # Calculate the number of patches needed in both the x and y directions with the best overlap value
    x_patches = (width - best_overlap) // (256 - best_overlap)
    y_patches = (height - best_overlap) // (256 - best_overlap)

    # Loop through all the patches and extract them from the input image
    patch_stack = []
    for y in range(y_patches):
        for x in range(x_patches):
            left = x * (256 - best_overlap)
            top = y * (256 - best_overlap)
            right = left + 256
            bottom = top + 256
            patch = img[left:right, top:bottom]
            patch_stack.append(patch)
            return patch_stack
            # do something with the patch here, like save it to a new file or process it further
            
test_image = np.random.randint(0, 255, size=(765, 787), dtype=np.uint8)
patches = divide_image(test_image)
len(patches)


1

## Why `beihai_adj_gaoqiazhen17` doesn't have S2 Coverage

In [14]:
import ee
import geemap
from utils.ee_utils import *
from utils.utils import *

In [ ]:
ee.Initialize()
Map = geemap.Map()

In [4]:
from rois import ROIS
rois = ROIS()
gaoq = rois.beihai_adj_gaoqiazhen

In [19]:
s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filterDate('2017-06-01', '2017-12-28') \
                    .filterBounds(gaoq)
s2 = s2.map(lambda img: img.set('roi_cloud_cover', get_mask_ones_ratio(get_cloud_mask(img)[2]))) 

There are s2 imgaes, the problem was the, overwriting of the `s2` collection, fixed it by changing main col name to `S2_COL`

In [20]:
sen2_print(s2)

,Property,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
0,system:time_start,2017-06-07,2017-06-07,2017-06-27,2017-06-27,2017-07-12,2017-07-12,2017-07-17,2017-07-17,2017-08-01,2017-08-01,2017-08-06,2017-08-06,2017-08-21,2017-08-21,2017-08-26,2017-08-26,2017-09-10,2017-09-10,2017-09-15,2017-09-15,2017-09-30,2017-09-30,2017-10-05,2017-10-05,2017-10-20,2017-10-20,2017-10-25,2017-10-25,2017-10-30,2017-10-30,2017-11-09,2017-11-09,2017-11-14,2017-11-14,2017-11-19,2017-11-19,2017-11-29,2017-11-29,2017-12-04,2017-12-04,2017-12-09,2017-12-09,2017-12-19,2017-12-19,2017-12-24,2017-12-24
1,roi_cloud_cover,49.148891,84.852904,67.459962,86.442563,74.627158,58.721023,88.587961,89.593764,93.330148,58.446223,14.233115,30.015909,0.974366,1.289893,73.836932,26.917047,77.223035,94.684324,47.366465,87.164193,98.165401,81.105902,89.520108,82.763577,0.656766,7.848514,0.482248,1.943341,1.721710,0,95.308576,93.697931,99.988553,96.211443,99.999917,95.703398,70.894340,53.976845,10.362358,6.703528,0.024967,3.316678,0,0,15.499368,7.966795
2,CLOUDY_PIXEL_PERCENTAGE,46.681600,83.527000,66.370800,88.649800,68.404500,55.540400,86.602300,91.213900,89.988700,55.771600,13.326900,29.198900,2.295500,1.075100,70.732900,26.887900,73.862500,96.888000,45.029400,89.388900,92.161800,82.778200,88.020800,82.264400,0.586700,7.357200,0.441300,1.851700,1.673700,0,99.996000,92.812500,99.263500,99.929200,99.901200,100,67.291500,51.351100,9.709100,6.792900,0.020400,3.139100,0,0,14.651300,7.937800
3,CLOUD_SHADOW_PERCENTAGE,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,VEGETATION_PERCENTAGE,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,NOT_VEGETATED_PERCENTAGE,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,CLOUD_COVERAGE_ASSESSMENT,46.681600,83.527000,66.370800,88.649800,68.404500,55.540400,86.602300,91.213900,89.988700,55.771600,13.326900,29.198900,2.295500,1.075100,70.732900,26.887900,73.862500,96.888000,45.029400,89.388900,92.161800,82.778200,88.020800,82.264400,0.586700,7.357200,0.441300,1.851700,1.673700,0,99.996000,92.812500,99.263500,99.929200,99.901200,100,67.291500,51.351100,9.709100,6.792900,0.020400,3.139100,0,0,14.651300,7.937800
7,GENERATION_TIME,1496805408000,1496805408000,1498533238000,1498533238000,1499829240000,1499829240000,1500261241000,1500261241000,1501557239000,1501557239000,1501989314000,1501989314000,1503285099000,1503285099000,1503717299000,1503717299000,1505013293000,1505013293000,1505445309000,1505445309000,1506741285000,1506741285000,1507173313000,1507173313000,1508469402000,1508469402000,1508920549000,1508920549000,1509349792000,1509349792000,1510209750000,1510209750000,1510648239000,1510648239000,1511187040000,1511187040000,1511937707000,1511937707000,1512376171000,1512376171000,1512799265000,1512799265000,1513665818000,1513665818000,1514104287000,1514104287000
8,SENSING_ORBIT_NUMBER,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75,75
9,NODATA_PIXEL_PERCENTAGE,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
